In [1]:
# Import dependencies
import pandas as pd
import numpy as np

In [2]:
# Import data
data_to_load = "Resources/purchase_data.csv"

In [3]:
# Convert to dataframe
purchase_data_df = pd.read_csv(data_to_load)

In [4]:
# Preview dataframe
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
# Check for holes in data
purchase_data_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

## Player Count
Total Number of Players

In [6]:
total_players = purchase_data_df["SN"].nunique()
total_players

576

## Purchasing Analysis (Total)
* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue

In [7]:
# Calculate number of unique items
unique_item_count = purchase_data_df["Item ID"].drop_duplicates().count()
unique_item_count

183

In [8]:
# Calculate average purchase price
average_purchase_price = round(purchase_data_df["Price"].mean(),2)
average_purchase_price

3.05

In [9]:
# Calculate total number of purchases
total_purchase_count = purchase_data_df["Purchase ID"].count()
total_purchase_count

780

In [10]:
# Calculate total revenue
total_revenue = purchase_data_df["Price"].sum()
total_revenue

2379.77

In [11]:
# Build dataframe
purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [unique_item_count],
                                      "Average Purchase Price": [average_purchase_price],
                                      "Total Number of Purchases": [total_purchase_count],
                                      "Total Revenue": [total_revenue]})
purchasing_analysis_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [12]:
# Identify exact syntax of Gender data types
purchase_data_df["Gender"].unique()

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [147]:
# Create Gender Demographics dataframe by omitting duplicate SN and grouping by Gender
gender_df = purchase_data_df[["Gender","SN"]]
gender_df = gender_df.drop_duplicates(["Gender", "SN"]).groupby("Gender").count()
gender_df = gender_df.rename(columns= {"SN": "Total Players"})
gender_df

,Total Players
Gender,
Female,81
Male,484
Other / Non-Disclosed,11


In [148]:
# Add Count of Male Players column
gender_df["Percentage of Players"] = round((gender_df["Total Players"]/total_players)*100,2)
gender_df

,Total Players,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)
The below each broken by gender:
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Gender

In [149]:
# Create new dataframe with unique players broken down by gender to start
gender_analysis_df = gender_df

In [150]:
# Add purchase count by gender column
gender_analysis_df["Total Purchase Count"] = purchase_data_df[["Purchase ID", "Gender"]].groupby("Gender").count()
gender_analysis_df

,Total Players,Percentage of Players,Total Purchase Count
Gender,,,
Female,81,14.06,113
Male,484,84.03,652
Other / Non-Disclosed,11,1.91,15


In [151]:
# Add average purchase price by gender column
gender_analysis_df["Average Purchase Price"] = round(purchase_data_df[["Price", "Gender"]].groupby("Gender").mean(),2)
gender_analysis_df

,Total Players,Percentage of Players,Total Purchase Count,Average Purchase Price
Gender,,,,
Female,81,14.06,113,3.20
Male,484,84.03,652,3.02
Other / Non-Disclosed,11,1.91,15,3.35


In [152]:
# Calculate total purchase value by gender column
gender_analysis_df["Total Purchase Value"] = purchase_data_df[["Price", "Gender"]].groupby("Gender").sum()
gender_analysis_df

,Total Players,Percentage of Players,Total Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,,,
Female,81,14.06,113,3.20,361.94
Male,484,84.03,652,3.02,1967.64
Other / Non-Disclosed,11,1.91,15,3.35,50.19


In [153]:
# Add Average Total Purchase per Person column and remove Percentage of Players column
gender_analysis_df["Avg Total Purchase per Person"] = round(gender_analysis_df["Total Purchase Value"]/
                                                            gender_analysis_df["Total Players"],2)
gender_analysis_df = gender_analysis_df.drop(columns =["Total Players", "Percentage of Players"])
gender_analysis_df

,Total Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


## Age Demographics
The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Average Purchase Total per Person by Age Group

In [154]:
# Create age demographics dataframe including duplicate SNs 
# to calculate purchase count, average purchase price, & total purchase value
age_bins_df = purchase_data_df[["Age", "SN", "Price"]]
age_bins_df.head()

,Age,SN,Price
0,20,Lisim78,3.53
1,40,Lisovynya38,1.56
2,24,Ithergue48,4.88
3,24,Chamassasya86,3.27
4,23,Iskosia90,1.44


In [155]:
# Create age demographics dataframe dropping duplicate SN to calculate average purchase total per person by age group
age_bins_df2 = age_bins_df[["Age", "SN"]].drop_duplicates("SN")
age_bins_df2.head()

,Age,SN
0,20,Lisim78
1,40,Lisovynya38
2,24,Ithergue48
3,24,Chamassasya86
4,23,Iskosia90


In [156]:
# Determine lowest figure in bin range
age_bins_df["Age"].min()

7

In [157]:
# Determine highest figure in bin range
age_bins_df["Age"].max()

45

In [158]:
# Create age bins in four year intervals and labels
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
labels = ["under 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "over 40"]

In [159]:
# Bin according to Age Range
age_bins_df["Age Range"]=pd.cut(age_bins_df["Age"], age_bins, labels = labels)

/Users/ajcascella/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [160]:
# Create age breakdown dataframe with bins
age_breakdown = age_bins_df2
age_breakdown["Age Range"]=pd.cut(age_bins_df2["Age"], age_bins, labels = labels)

# Organize by Age Range
age_breakdown = age_breakdown.groupby("Age Range").count()

# Rename Age column
age_breakdown = age_breakdown.rename(columns = {"Age": "Total Players"})

# Drop SN column
age_breakdown = age_breakdown.drop(columns = "SN")

# Create Percentage of Players column
age_breakdown["Percentage of Players"] = round((age_breakdown["Total Players"]/total_players)*100,2)
age_breakdown

,Total Players,Percentage of Players
Age Range,,
under 10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
over 40,12,2.08


In [161]:
# Calculate total purchase count by age group
purchase_count_per_bin =age_bins_df.groupby("Age Range").count()["Age"]
purchase_count_per_bin

Age Range
under 10     23
10-14        28
15-19       136
20-24       365
25-29       101
30-34        73
35-39        41
over 40      13
Name: Age, dtype: int64

In [162]:
# Calculate average purchase price
purchase_avg_per_bin= round(total_purchases_per_bin/purchase_count_per_bin,2)
purchase_avg_per_bin

Age Range
under 10    3.35
10-14       2.96
15-19       3.04
20-24       3.05
25-29       2.90
30-34       2.93
35-39       3.60
over 40     2.94
dtype: float64

In [163]:
# Calculate total purchase sum by age group
total_purchases_per_bin= age_bins_df.groupby("Age Range")["Price"].sum()
total_purchases_per_bin

Age Range
under 10      77.13
10-14         82.78
15-19        412.89
20-24       1114.06
25-29        293.00
30-34        214.00
35-39        147.67
over 40       38.24
Name: Price, dtype: float64

In [164]:
# Calculate average total purchase per person using age breakdown dataframe
average_total_price_per_bin = round(total_purchases_per_bin/age_breakdown["Total Players"],2)

In [165]:
# Create dataframe
age_spending_analysis = pd.DataFrame({"Purchase Count": (purchase_count_per_bin),
                                     "Average Purchase Price": purchase_avg_per_bin,
                                     "Total Purchase Value":total_purchases_per_bin,
                                     "Avg Total Purchase per Person":average_total_price_per_bin})
age_spending_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
under 10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
over 40,13,2.94,38.24,3.19


## Top Spenders
Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [230]:
# Create Top Spenders dataframe
top_spenders_df = purchase_data_df[["SN", "Price"]].groupby("SN").sum()["Price"]
top_spenders_df = top_spenders_df.nlargest(5)
top_spenders_df = pd.DataFrame(top_spenders_df)
top_spenders_df

,Price
SN,
Lisosia93,18.96
Idastidru52,15.45
Chamjask73,13.83
Iral74,13.62
Iskadarya95,13.10


In [231]:
# Rename Price column to be Total Purchase Value
top_spenders_df = top_spenders_df.rename(columns= {"Price": "Total Purchase Value"})
top_spenders_df

,Total Purchase Value
SN,
Lisosia93,18.96
Idastidru52,15.45
Chamjask73,13.83
Iral74,13.62
Iskadarya95,13.10


In [242]:
# Create Purchase Count column
top_spenders_df["Purchase Count"] = purchase_data_df.loc[purchase_data_df["SN"] == top_spenders_df["SN"]].count()
top_spenders_df

KeyError: 'SN'

In [228]:
# Create Average Purchase Price
top_spenders_df["Average Purchase Price"] = top_spenders_df["Total Purchase Value"]/top_spenders_df["Purchase Count"]

In [229]:
top_spenders_df

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,18.96,780,0.024308
Idastidru52,15.45,780,0.019808
Chamjask73,13.83,780,0.017731
Iral74,13.62,780,0.017462
Iskadarya95,13.10,780,0.016795
